# Note về transformer ở đây

- Đầu vào ban đầu là video có kích thước: $video: (3\times F\times H\times W)$ với $H, W$ là kích thước video, $F$ là độ dài video 

### 1. Bước Video Imbedding
- Sau khi đi qua mạng backbone rút trích đặc trưng: $video\space feature: (C\times t\times h\times w)$:
    + Lúc này đặc trưng đã trích lọc thông tin và giảm đáng kể so với video gốc với: $t < F, h \ll H, w \ll W$

### 2. Fourier Positional Encoder
- Tạo ra đặc trưng Fourier Positional Encoder sẽ được viết cụ thể sau, tổng quan là: thực hiện thêm thông tin về vị trí của các đặc trưng (cụ thể là $t\times h\times w$ cho mỗi đặc trưng trong kênh $C$ được rút từ backbone - tựa như cho mỗi từ trong NLP) thông qua một phép biến đổi tương tự Fourier Transformer, giúp đặc trưng dù có bị tác động bởi một một góc $\theta$ (bị xê dịch) cũng sẽ phụ thuộc vào model, chứ không phụ thuộc vào vị trí. [Fourier Positional Encoder](https://sair.synerise.com/fourier-feature-encoding/?fbclid=IwAR1Zti_AmzXHVxBgZGk2NaGb8Q-ot1tm0Ax2okX_Q84ovf3Y-cCTnKwLpyU)
- Mỗi giá trị vị trí sẽ được nhân với các tần số khác nhau, để lưu trữ vị trí trong tần số. $t\times h\times w\times numFreq$
- Concat với đặc trưng ta được một tensor mới với kích thước: $t\times h\times w\times (numFreq + C)$

Trong bài báo:
- Reshape lại đặc trưng thành $video\space feature: (C\times (thw))$, reshape position encoder $n\times(twh)$
Thực tế:
- Concat lại r mới reshape thành: $thw\times D$ với $D = numFreq + C$


### 3. Tới bước Cross Attention
- Transformer original: $ attetion(Q, K, V) = softmax (QK^T)V $, $Q$ và $K$ và $V$ sẽ được tính dựa trên input: $Q = X\times W_Q$ và kích thước lần lượt là $Q, K: (n, d_k), V:(n, d_v)$, cụ thể trong bản gốc của google ![Transformer](media/transformer_original.png "")
    + Với phép toán như trên, ta sẽ có độ phức tạp của attention phiên bản originnal là: $n\times d_k \times n + n \times n + n \times n \times d_v$
    + nếu bỏ qua số chiều $d_k, d_v$ thường là của vector đặc trưng, ta được độ phức tạp của attention original: $O(n^2)$

- Transformer Upgrade: Đã có rất nhiều phiên bản dược sinh ra để cải tiến vấn đề time quadratic complexity (sẽ liệt kê khi đi vào viết khóa luận), ở đây Tempr4 sử dụng Efficient attetion: Perceiver Achitecture để giải quyết vấn đề này:
    + Cụ thể: kiến trúc đề xuất với K (key) và V (value) là phép chiếu trực tiếp từ input array: $X (n, d_X) \times W_K (d_X, d_K) = K (n, d_K)$